In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Paso 1: Cargar los Datos Consolidados

- Cargamos el archivo CSV generado en la libreta 1.
- Aseguramos que el índice sea de tipo `datetime`.

In [3]:
HEATID = '6F0020'
filename = f'consolidated_timeseries_{HEATID}.csv'

try:
    df = pd.read_csv(filename, index_col=0, parse_dates=True)
    print(f"Datos cargados exitosamente desde {filename}")
    print("Dimensiones del dataframe:", df.shape)
    print("Columnas:", df.columns.tolist())
except FileNotFoundError:
    print(f"Error: El archivo {filename} no fue encontrado. Asegúrate de ejecutar la primera libreta primero.")

Datos cargados exitosamente desde consolidated_timeseries_6F0020.csv
Dimensiones del dataframe: (52, 16)
Columnas: ['HEATID', 'Total_Material_Added', 'V', 'P', 'S', 'C', 'MW', 'TAP', 'O2_used', 'GAS_used', 'O2_FLOW', 'GAS_FLOW', 'Carbon_used', 'INJ_FLOW_CARBON', 'TEMP', 'VALO2_PPM']


# Paso 2: Preparación de Datos para el LSTM

- **Escalado:** Normalizamos todas las características al rango `[0, 1]` usando `MinMaxScaler`. Esto es crucial para el buen rendimiento de las redes neuronales.
- **Creación de Secuencias:** Convertimos la serie de tiempo en secuencias de entrada (`X`) y etiquetas de salida (`y`) usando una ventana deslizante. El modelo usará una secuencia de `N` pasos de tiempo para predecir el paso `N+1`.


In [5]:
# Seleccionar solo columnas numéricas para escalar
df_numeric = df.select_dtypes(include=np.number)

# Escalar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_numeric)

# Crear secuencias (ventana deslizante)
def create_sequences(data, time_steps=10):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

TIME_STEPS = 10  # Usaremos los últimos 10 minutos para predecir el siguiente
X, y = create_sequences(scaled_data, TIME_STEPS)

print(f"Datos convertidos en secuencias:")
print("Forma de X (entradas):", X.shape) # (Muestras, Pasos de tiempo, Características)
print("Forma de y (salidas):", y.shape)   # (Muestras, Características)

Datos convertidos en secuencias:
Forma de X (entradas): (42, 10, 15)
Forma de y (salidas): (42, 15)


# Paso 3: Construir el Modelo Stacked LSTM
 
- Creamos un modelo secuencial en Keras.
- Añadimos varias capas `LSTM`. La opción `return_sequences=True` es necesaria en las capas intermedias para pasar la secuencia completa a la siguiente capa.
- Usamos `Dropout` para prevenir el sobreajuste.
- La capa final es `Dense` y tiene tantas neuronas como características queremos predecir.


In [6]:
model = Sequential()

# Primera capa LSTM
model.add(LSTM(units=128, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))

# Segunda capa LSTM
model.add(LSTM(units=64, return_sequences=False))
model.add(Dropout(0.2))

# Capa de salida
model.add(Dense(units=X.shape[2]))

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()

C:\Users\rprie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10, 128)        │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 124,111 (484.81 KB)

 Trainable params: 124,111 (484.81 KB)

 Non-trainable params: 0 (0.00 B)

# Paso 4: Entrenar el Modelo
 
- Entrenamos el modelo con las secuencias `X` e `y`.
- **Importante:** En un caso real, entrenaríamos el modelo con datos de *muchos* `HEATID`s considerados "normales". Aquí, como ejemplo, estamos entrenando y probando con el mismo `HEATID`.


In [7]:
print("\nEntrenando el modelo...")
history = model.fit(
    X, y,
    epochs=50,       # Número de veces que el modelo ve todos los datos
    batch_size=32,   # Número de muestras por actualización de gradiente
    validation_split=0.1, # Usar 10% de los datos para validación
    shuffle=False    # Importante para series de tiempo no barajar los datos
)
print("Entrenamiento completado.")


Entrenando el modelo...
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 338ms/step - loss: 0.1337 - val_loss: 0.1408
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 338ms/step - loss: 0.1337 - val_loss: 0.1408
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.1127 - val_loss: 0.1219
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.1127 - val_loss: 0.1219
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.0994 - val_loss: 0.1046
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.0994 - val_loss: 0.1046
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0901 - val_loss: 0.0888
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0901 - val_loss: 0.0888
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0861 - val_loss: 0.0747
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0861 - val_loss: 0.0747
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.0786 - val_loss: 0.0623
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.0

# Paso 5: Detección de Anomalías
 
- Usamos el modelo entrenado para hacer predicciones sobre los datos de entrenamiento.
- Calculamos el **error de reconstrucción** (en este caso, el Error Absoluto Medio - MAE) entre las predicciones y los valores reales.
- Asumimos que los errores en los datos normales siguen una distribución gaussiana.
- Establecemos un **umbral de anomalía** (ej. media del error + 3 desviaciones estándar).
- Cualquier error que supere este umbral se considera una anomalía.

In [8]:
# Calcular el error de reconstrucción
predictions = model.predict(X)
mae_loss = np.mean(np.abs(predictions - y), axis=1)

# Determinar el umbral de anomalía
threshold = np.mean(mae_loss) + 3 * np.std(mae_loss)
print(f"\nUmbral de anomalía (Error de Reconstrucción): {threshold}")

# Identificar anomalías
anomalies_indices = np.where(mae_loss > threshold)[0]
print(f"Se encontraron {len(anomalies_indices)} posibles anomalías.")

# Crear un dataframe con los resultados
df_result = df.iloc[TIME_STEPS:].copy()
df_result['loss'] = mae_loss
df_result['threshold'] = threshold
df_result['anomaly'] = df_result['loss'] > df_result['threshold']

anomalies = df_result[df_result['anomaly']]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step

Umbral de anomalía (Error de Reconstrucción): 0.27160516654724237
Se encontraron 1 posibles anomalías.

Umbral de anomalía (Error de Reconstrucción): 0.27160516654724237
Se encontraron 1 posibles anomalías.
